In [1]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_verbose

set_verbose(True)
model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("說出一個關於'{topic}'的笑話")

chain = prompt | model | StrOutputParser()

In [3]:
chain.invoke({"topic": "軟體工程師"})

'為什麼軟體工程師總是在午夜時分工作？\n因為他們喜歡在「黑暗中」編程！'

In [7]:
from langchain_openai import ChatOpenAI
from langchain.globals import set_verbose
from langchain_core.prompts import ChatPromptTemplate

system = """你是一個專業的心理學家，你擅長分析日記中所包含的情緒。 \
你將需要根據使用者的日記內容分析其情緒，並將其代表的顏色以及對應的內容段落，分別放進名為 "color" 和 "content" 的 key 中，並以 JSON 格式回傳。 \
注意：你最少需要挑選一個顏色，並且最多挑選不超過兩個顏色，請挑選日記中最具有代表性的段落。 \
以下是 8 種不同的情緒所對應的顏色: \
1. 憤怒暴躁: Red \
2. 快樂喜悅: Yellow \
3. 悲傷難過: Blue \
4. 恐懼害怕: Purple \
5. 焦慮不安: Orange \
6. 厭惡煩躁: Green \
7. 平靜祥和: Indigo \
8. 其他 (當日記不包含任何情緒或是沒有內容時): Gray \

以下是幾個範例:

example_user: 今天的工作非常緊張，一整天都在忙碌中度過。午休時，莉莉邀請我週末一起去看電影，我感到非常開心。下班後，坐公交回家，一路上都在想著週末的計劃。回到家後，煮了一些意大利麵，和臭臭一起享用。今天牠的狀態很好，我也感到很放鬆。
example_assistant: [{"color": "Yellow", "content": "午休時，莉莉邀請我週末一起去看電影，我感到非常開心。"}, {"color": "Indigo", "content": "回到家後，煮了一些意大利麵，和臭臭一起享用。今天牠的狀態很好，我也感到很放鬆。"}]

example_user: 今天沒有發生有趣的事
example_assistant: [{"color": "Gray", "content": "今天沒有發生有趣的事"}]

example_user: (使用者沒有輸入任何內容)
example_assistant: [{"color": "Gray", "content": ""}]"""


ValueError: Unsupported function

{'type': 'array', 'items': {'type': 'object', 'properties': {'color': {'type': 'string', 'description': '日記內容所對應的顏色', 'enum': ['Red', 'Yellow', 'Blue', 'Purple', 'Orange', 'Green', 'Indigo', 'Gray'], 'default': 'Gray'}, 'content': {'type': 'string', 'description': '片段的日記內容'}}, 'required': ['color', 'content'], 'title': '顏色物件'}, 'description': '將使用者的日記內容依照情緒分類，並回傳對應的顏色及日記內容', 'title': '分類情緒'}

Functions must be passed in as Dict, pydantic.BaseModel, or Callable. If they're a dict they must either be in OpenAI function format or valid JSON schema with top-level 'title' and 'description' keys.

In [4]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.globals import set_verbose

set_verbose(True)

class EmotionAnalysis(BaseModel):
    color: str = Field(description="Emotion color based on the diary content")
    content: str = Field(description="Content of the diary representing the emotion")

class EmotionAnalysisList(BaseModel):
    colors: List[EmotionAnalysis]  

model = ChatOpenAI(model="gpt-3.5-turbo-0125")


parser = JsonOutputParser(pydantic_object=EmotionAnalysisList)

prompt = PromptTemplate(
    template="""你是一個專業的心理學家，你擅長分析日記中所包含的情緒。 \
                你需要根據使用者的日記內容分析其情緒，並挑選其代表的顏色以及對應的內容段落。 \
                注意：每個段落不要超過 100 字，同時不要回傳日記以外的內容。 \
                限制：你最少需要挑選一個顏色，並且最多挑選不超過兩個顏色，請挑選日記中最具有代表性的段落。 \
                以下是 8 種不同的情緒所對應的顏色: \
                1. 憤怒暴躁: Red \
                2. 快樂喜悅: Yellow \
                3. 悲傷難過: Blue \
                4. 恐懼害怕: Purple \
                5. 焦慮不安: Orange \
                6. 厭惡煩躁: Green \
                7. 平靜祥和: Indigo \
                8. 其他 (當日記不包含任何情緒或是沒有內容時): Gray \

                {format_instructions}

                {query}
            """,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

diary_entry = "今天一早出門時，感覺有些不安。工作中遇到一些困難的問題，但在與同事的共同努力下最終解決了。"
output = chain.invoke({"query": diary_entry})

print(output['colors'][:2])


[{'color': 'Orange', 'content': '今天一早出門時，感覺有些不安。工作中遇到一些困難的問題，但在與同事的共同努力下最終解決了。'}]


In [ ]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.globals import set_verbose

set_verbose(True)

class EmotionAnalysis(BaseModel):
    color: str = Field(description="Emotion color based on the diary content")
    content: str = Field(description="Content of the diary representing the emotion")

class EmotionAnalysisList(BaseModel):
    emotion_colors: List[EmotionAnalysis]  

model = ChatOpenAI(model="gpt-3.5-turbo-0125")


parser = JsonOutputParser(pydantic_object=EmotionAnalysisList)

prompt = PromptTemplate(
    template="""You are a professional psychologist specializing in analyzing emotions contained in diaries.\
        Based on the user's diary content, you need to analyze their emotions and place the representative colors and corresponding content paragraphs into the keys named "color" and "content" respectively, and return it in JSON format.\
        Note: You must select at least one color and no more than two colors. Please select the most representative paragraphs in the diary.\
Here are the colors corresponding to 8 different emotions:
1. Anger/Irritability: Red
2. Happiness/Joy: Yellow
3. Sadness: Blue
4. Fear: Purple
5. Anxiety: Orange
6. Disgust/Irritation: Green
7. Calm/Peacefulness: Indigo
8. Other (when the diary does not contain any emotions or has no content): Gray

{format_instructions}

{query}""",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

diary_entry = "今天是無聊的一天"
output = chain.invoke({"query": diary_entry})

print(output)


{'emotion_colors': [{'color': 'Gray', 'content': 'Today is a boring day.'}]}
